### Linear regression model with a single predictor

All the examples of linear regression models we have seen today share a common form: 
$$
Y_i = \beta_0 + \beta_1 X_i + \epsilon_i
$$
$$
\epsilon_i \sim N(0, \sigma)
$$

where
- $\beta_0$ and $\beta_1$ are some fixed numbers
- $\epsilon_i$ are independent, normally distributed variables with $E(\epsilon_i)=0$ and $SD(\epsilon_i)=\sigma$, where $\sigma$ is a strictly positive number.

**Model parameters**: The models differ by assigning different values to the parameters
- $\beta_0, \beta_1$: the regression coefficients
- $\sigma$: the standard deviation of the residua


A statistical model with this form is called a **linear regression model** with a single predictor (or **univariate linear regression**).

## Key assumptions of the linear regression model

1) __Linearity:__ $Y_i = \beta_0 + \beta_1 X_i + \epsilon_i,\ E(\epsilon_i)=0$
2) __Normally distributed residuals:__ The residuals $\epsilon_i$ are normally distributed.
3) __Homoscedasticity:__ The standard deviation of the residuals does not depend on the value of $X_i$.
4) __Independence:__ The residuals $\epsilon_i$ are independent.

In [ ]:
## ols model
est_mod = ols('y~bmi+sex+age', data=diab).fit()
est_mod.params.round(2)

In [ ]:
## if need to **2/**3 
est_mod = ols('pid7~urbancity+age+I(age**2)+I(age**3)', data=fc).fit()
est_mod.params.round(2)

In [ ]:
## see p and 95%
est_mod.summary().tables[1]

In [ ]:
diab_estmod['exp_y'] = est_mod.predict() # this function computes E(Y) adding a new column in dataframe

## Logistic Regression

In [ ]:
logm = smf.logit('Survived~Fare',data=dft).fit()

In [ ]:
# estimate a logistic regression using categorical variables.
modelp = smf.logit('Survived~C(Pclass)',data=dft).fit()

In [ ]:
## Interaction Terms in Logistic Regressions
modelp = smf.logit('Survived~Sex+Fare+Sex*Fare',data=dft).fit()
modelp.params

## interpretation 
#The log-odd of survival will be 2.099345 lower when sex=male compared to female; 
#The log-odd of survival will be 0.019878 higher when Fareprice increase by 1   
# the estimate for the interaction term means how much the effect of sex/fareprice on survival depends on each other. 
#in this case, the coefficient is nefative, so we can say for example the effect of fareprice on survival decreases when a person is a male.  

#### Language for Interpretation

$\beta_1$ : The amount by which we expect the outcome $Y$ to increase for each 1-unit increase in $X_1$

$\beta_2$ : The amount by which we expect the outcome $Y$ to increase for each 1-unit increase in $X_2$

95% confidence interval: The range within which we would expect the true value of a parameter. If multiple samples were drawn from the same population and a 95% CI calculated for each sample, we would expect the true parameter to be found within 95% of these CIs.

Interaction coefficient: A statistically significant interaction terms asserts that the relationship between $X_1$ and $Y$ is conditional upon another variable $X_2$. In this case, the relationship / slope depends on the value of $X_2$

In [ ]:
import pandas as pd
from statsmodels.formula.api import ols

In [ ]:
# build ols model to Estimate the average treatment effect

mod = ols(formula='y ~ treatment', data=df)
res = mod.fit(cov_type = "HC0")
res.summary()

In [ ]:
# Check the random assignment assumption by checking covariate balance on age(one covariate) (i.e. run a linear regression of age on treatment
mod = ols(formula='age ~ treatment', data=df)
res = mod.fit(cov_type = "HC0")
res.summary()

In [ ]:
# or use stargazer:
from stargazer.stargazer import Stargazer

mod = smf.ols(formula='age ~ treatment', data=df).fit(cov_type='HC0') 
Stargazer([mod])

从dataframe中截取部分数据


In [ ]:
df_pretrend =  df[(df.time >= -8) & (df.time <= 12)].copy()

create dummy

In [ ]:
df_pre = pd.get_dummies(df_pre,columns=["time"])

the coefficient on treatment means the difference in age between the treatment and control group


In [ ]:
df.groupby('treatment')['age'].mean()[1]-df.groupby('treatment')['age'].mean()[0]

The standard way of estimating an average treatment effect under conditional random assignment is to:  
add the conditioning variable (in this case glasses) to a linear regression alongside the treatment indicator.

In [ ]:
mod = ols(formula='y ~ treatemnt + glasses', data=df)
res = mod.fit(cov_type = "HC0")
res.summary()

In [ ]:
# check the conditional random assignment (CIA) assumption by checking covariate balance conditional on glass wearing
mod = smf.ols(formula='age ~ treatemnt + glasses', data=df)
res = mod.fit(cov_type = "HC0")
res.summary()

random assignment of the treatment (seeing obama) seems to hold (the coefficient on obama is small and not statistically significant)

In [ ]:
## do covariate balance check on a list of variables:

#Defining a list of predetermined variables
covariates = ['age', 'children', 'city', 'income', 'yrs_sin_start']
models = []
for var in covariates:
    mod = ols(formula=f'{var} ~ D ', data = df).fit(cov_type='HC0') 
    models.append(mod)
    
    
table_result = Stargazer(models)

#Adding the variable names to each model
table_result.custom_columns(covariates, [1 for i in range(len(covariates))])
table_result

In [ ]:
# 2SLS model:

# Note the order of the inputs
# 1. the outcome variable (Y)
# 2. exog has to hold our constant and any controls that we might add (here no controls)
# 3. endog will hold our treatment variable (D)
# 4. intruments will hold  our instrument variable (Z)

df['const'] = 1
ivmod = IV2SLS(df['Y'], exog = df['const'] ,endog=df['D'],instruments=df['Z']).fit(cov_type='robust')
print(ivmod)


if need to add control variable in the model:


In [ ]:
df['const'] = 1
ivmod = IV2SLS(df['Y'], exog =  df[['const', 'control_va']]  ,endog=df['D'],instruments=df['Z']).fit(cov_type='robust')
print(ivmod)

In [ ]:
#95% CI
ivmod.conf_int()

In [ ]:
## obtain appropriately clustered standard errors
## for each of the ATEs 

mod3 = smf.ols(formula='cases ~ lockdown*t2', data=df0_2).fit(cov_type='cluster', cov_kwds={'groups': df0_2['muni_name']})
mod3.summary()

Creating the time dummies  
Creating the interactions

In [ ]:
## 为了保证原来的time column不丢失，要在新的dataframe里面把time变成dummy，最后再加回原来的dataframe中

# Creating the time dummies
df_pretrend_dum = pd.get_dummies(df_pretrend, columns=['time']).drop("time_0", axis = 1)

# Creating the interactions
cols = df_pretrend_dum.loc[:, 'time_1':'time_12'].columns
df_pretrend_dum[list(map(lambda x: x+"xtreat", cols))] = df_pretrend_dum[cols].apply(lambda x: x*df_pretrend_dum['lockdown'])

print(df_pretrend_dum.shape)
df_pretrend_dum.head()

In [ ]:
# set 0 as the reference in the regression model(when including catagory variables, the first one is always set as the reference)
mod = ols(formula='cases ~ lockdown*C(time, Treatment(reference=0))', data=df_pre)
res = mod.fit(cov_type='cluster', cov_kwds={'groups': df_pre['muni_id']})
res.summary().tables[1]

supervised learning


In [ ]:
##  sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
ols = LinearRegression()
#fitting the model
x = df.drop('y',axis=1)
y = df['y']
model_skl = ols.fit(x, y)

splitting data into training/testing sets

In [ ]:
X = df.drop('y',axis=1)

y = df['y']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [ ]:
ols = LinearRegression()
model = ols.fit(x_train, y_train)
# estimate y using the model
yhat_test = model.predict(x_test)

In [ ]:
## calculation of MSE
from sklearn.metrics import mean_squared_error 

# Compute using the predicted and actual values
mean_squared_error(y_test, yhat_test)

In [ ]:
## RMSE
mean_squared_error(y_test, yhat_test, sample_weight=None,squared=False)


### polynomial regression

polynomial regression approach (of degree k = 6). Remember that a polynomial regression of degree k = 6 just means estimating a linear regression like the following using OLS:

$Y_i =\beta_0 +\beta_1X_i + \beta_2X_i^2 +\beta_3X_i^3 +\beta_4X_i^4 +\beta_5X_i^5 +\beta_6X_i^6 + ε_i$

First we can repeat our supervised learning exercise using polynomial regression on the first data set.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

X=df.iloc[:,1:]
y=df.y

# Tell the function the number of polynomials that you with to obtain and for your x
newx = PolynomialFeatures(6).fit_transform(X)

#Column names for transparency
cols = ['x_'+str(i) for i in range(2,7)]

#array to df
newdf=pd.DataFrame(newx, columns = ["const", "x"]+cols)
newdf.head()

In [ ]:
## split dataset

#defining x as all the variables in the dataframe besides from y
X = newdf.iloc[:, 1:]

#Splitting into the initial training & test data
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [ ]:
## fit the model
model_2 = ols.fit(x_train_2,y_train_2)

### Supervised learning using k Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

#save the knn function
knn = KNeighborsRegressor(n_neighbors=6)

In [ ]:
X = df.iloc[:,1:].copy()
x_train, x_test, y_train, y_test = train_test_split(X, df['y'], test_size = 0.2, random_state = 123)

#Fitting the model
knn.fit(x_train, y_train)

#Predicting
yhat_test = knn.predict(x_test)

In [ ]:
knn=KNeighborsRegressor()
params_knn = {'n_neighbors': [1,5,10,15,20,25,30,35,40,45,50,60,80,100,150,200,250,300,400,500]}
gscv1 = GridSearchCV(knn, params_knn, cv = 5, scoring='neg_mean_squared_error')
    
# reshape x:
X = df.drop('speed',axis=1)
y = df['speed']
        
# Split the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

#Fitting the model
gscv1.fit(x_train, y_train)

#Predicting
yhat_test = gscv1.predict(x_test)

---

## 4. Supervised learning, step-by-step

As discussed in the slides, a standard prediction/supervised learning analysis typically goes through the following steps:

    

1. Split the data into a training and a test part
2. Pick hyperparameters, often using cross-validation
3. Train the model on the training data
4. Asses model performance on the test data
5. (Re-estimate model on the full data)
6. Compute predictions in out-of-sample data

**Very briefly describe what each of the above steps entails and why they are necessary.**


We need to be able to compare how our model does out-of-sample (on data that it has never seen). Else we risk training a model that performs great on our own data but does poorly on unseen data - which would be a bummer when the goal is to train a model that is good at predicting. Key-word: we do not want to overfit.


We want to choose the best model and most models entail parameters that have to be set a priori. By hypertuning our model we can find the model specifications that best predict our target within our training sample. Cross-validation allows us to get a better feel of how our model will perform out of sample - our model is train on several train-test splits instead of one. By not saving the test set for last we will bias our model and once again face the risk of overfitting.

Example of 5-fold cv process:
![fold](https://www.pitcherlist.com/wp-content/uploads/1_NyvaFiG_jXcGgOaouumYJQ-768x241.jpeg)


We train our model on the training data, because we have to establish how to best predict our target with the features that we have, before we can apply our model to unseen data.

We asses model performance on test data, as we want to validate how our model(s) do(es) on out of sample data - check for overfitting etc.

We re-estimate our model on the full data as we want to train our final model on as much availible information as possible. 

We compute out-of-sample predictions in order to fufill the purpose of having trained our model. 


---

### hyperparameters using cross-validation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

In [ ]:
### example:   find the best K
from sklearn.neighbors import KNeighborsRegressor

# Create the knn function
knn = KNeighborsRegressor()
params_knn = {'n_neighbors': [1,5,10,15,20,25,30,35,40,45,50,60,80,100,150]}
gscv1 = GridSearchCV(knn, params_knn, cv = 5, scoring='neg_mean_squared_error')

# reshape x:
X = df.iloc[:,1:].copy()
# Split the data
x_train, x_test, y_train, y_test = train_test_split(X, df['y'], test_size = 0.2, random_state = 123)

#Fitting the model
gscv1.fit(x_train, y_train)
        
print(f'KNN based on data from sublearnbig2 ({df.shape[0]} obs): Best model: {gscv1.best_params_}, with mean (negative) MSE: {gscv1.best_score_.round(3)}')

---

## classification models